# Applications of Artificial Intelligence
## Big Data – MapReduce
### Setup
This example will show you how you can use Apache Spark with PySpark on a local machine. This won't allow you to benefit from the massive parallelism that makes this technique so powerful, but we can still demonstrate how some code would be written, so you could then apply this knowledge to a real cluster deployment across many machines.

Unlike the other examples, running this code will require you to install some new software, and since this will be platform dependant, this might require a bit of experimentation. If you'd rather not install any new software, you can still read the notebook and see the results below each cell.

### Installation
First you will need to install Apache Spark and PySpark, which may require you to install Java and/or Scala as well if you do not already. Try following the instructions on one of the sites listed below. 

*Note that we have not tested all of these resources, and are not responsible for the content of third-party websites. Please ensure you are comfortable with technical installations before making any modifications to your own machine. If you are unsure, ask for advice or simply read the notebook without installing anything.*

* Windows 10 – [https://phoenixnap.com/kb/install-spark-on-windows-10](https://phoenixnap.com/kb/install-spark-on-windows-10)
* macOS – [https://medium.com/swlh/pyspark-on-macos-installation-and-use-31f84ca61400](https://medium.com/swlh/pyspark-on-macos-installation-and-use-31f84ca61400)
* Linux – [https://computingforgeeks.com/how-to-install-apache-spark-on-ubuntu-debian/](https://computingforgeeks.com/how-to-install-apache-spark-on-ubuntu-debian/)

You do not necessarily need to set the environment variables or run Spark from the command line as the tutorials might get you to do – just get the software installed first of all. We are going to use another Python library called `findspark` to help us use Spark without any additional configuration (though if this does not work, return to the tutorials!).

To install `findspark`, use your normal Python package manager, e.g. `pip install findspark` or `conda install pyspark`.

Once that is done, the code below should run without error.

In [2]:
import findspark
findspark.init()

from pyspark import SparkContext

### PySpark Example
First, we must create a `SparkContext`. This is the entry point to any Spark functionality. When we run a Spark application a 'driver program' starts which has the main function, and your SparkContext gets initialised here. The driver program then sends operations to be run inside the executors on worker nodes. 

As we are leaving everything with its default settings, the local machine will also act as the worker node, but this is where you could configure Spark to run on a real cluster of multiple worker nodes.

In [3]:
sc = SparkContext()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/03/18 19:03:17 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Once our SparkContext is running, we can load some data that exists on the distributed file system. Again for our example we'll use a local text file from our machine. But we could just as easily pass in the location of a massive dataset on the distributed file system, where the actual data is split into chunks across the cluster.

We'll use the PySpark `.textFile(...)` method of the SparkContext object to load our file – a text document containing the entirety of War and Peace by Leo Tolstoy.

In [4]:
text_file = sc.textFile("war-and-peace.txt")
print(type(text_file))

<class 'pyspark.rdd.RDD'>


This method returns an object of type `RDD` which stands for Resilient Distributed Dataset. This is the fundamental data structure of Spark. An RDD is an immutable, distributed collection of objects. The `text_file` object in the cell above doesn't store the entire text file in memory, it is just a *reference* to the data which is distributed on different nodes of the cluster – or in our case, just stored on our machine. Operations are performed lazily, so nothing actually happens with this data until we try to access it.

An RDD can contain any type of Python objects, including user defined classes. 

We can look at the contents of our RDD using the `.take(n)` function, which will return a list of the first `n` elements – again this is evaluating lazily, it will only read as much from the file on disk as it actually needs to, depending on how many lines the user has requested.

In [5]:
print(text_file.take(1))

['The Project Gutenberg EBook of War and Peace, by Leo Tolstoy']


Spark makes use of RDD to achieve fast and efficient MapReduce operations. Because RDDs are immutable, when we apply transformations and actions to them this results in a new RDD object with the result of the transformation rather than an edit to the original RDD. In fact, due to the lazy evaluation, it is really just remembering the operations that it *needs* to perform once you actually request some results. 

We are going to count the number of times each word occurs in the file, as we showed in the unit example. So the basic idea is to perform a map which splits the text into words, another map which associates each word with the value `1`, then a reduce which combines these values per key.

Remember that a `map` operation itself performs any operation over the data, so we must pass a *function* in as a parameter. Python makes this pretty easy thanks to its `lambda` functions, but you could also pass in the name of an existing or custom function.

The operations are chained into one long sequence in the cell below, with some additional work to remove empty lines and convert all the words to lowercase. 

We start with `flatMap` because we are splitting strings into lists of strings, so we can have more than one output from a single input. `flatMap` collects all of the results into one flat structure. `.map` operations must have just one output for each input.

`filter` is another common functional programming higher-order function, which is being used here to split out any empty strings from the data, which might occur due to empty lines in the original file, or because of the way the words were split.

Then we use another `map` to associate each word as a key with a value of `1`. Notice that key-value pairs are simply tuples in Python.

Finally we reduce the data, which will combine key-value pairs that have the same key, and here we simply add the values together.

In [6]:
counts = text_file.flatMap(lambda text: text.lower().split()) \
         .filter(lambda word: word != '') \
         .map(lambda word: (word, 1)) \
         .reduceByKey(lambda a, b: a + b)

Running the cell above takes no time at all because, again, nothing is evaluated yet. The result of this operation, the `counts` variable, is another RDD. The data itself could be distributed across our cluster of nodes.

If we want to check the results we can move the data into the driver node by calling a function called `collect()`. We only want to use the `collect` function after we have run our analysis resulting in a smaller dataset, because the whole point is that we might not be able to store the original dataset in our local memory.

In fact, even though War and Peace would fit in memory in its entirety, we still don't want to print out every unique word in this notebook –  let's just look at the counts of the most frequent 10 words. 

In the cell below, we use `takeOrdered(...)` which allows us to take the top `n` items according to some ordering (sorting) from the RDD. The key is another lambda function: in this case defining how to order each item. We want to order by the *value* rather than the *key*, so we use element `1` rather than `0`, and we also want them in descending order, so we order by `-x[1]` rather than `x[1]`.

In [7]:
output = counts.takeOrdered(10, key = lambda x: -x[1])
for word, count in output:
    print(f"{word}: {count}")

the: 34262
and: 21403
to: 16502
of: 14903
a: 10414
he: 9297
in: 8607
his: 7932
that: 7417
was: 7202


So there we have the results of our analysis. The beauty of this approach is that this code which works on a single novel would scale up to a massive dataset containing billions of sentences – providing that we have a cluster large enough to store it, and with enough notes that we can process small chunks of the data in parallel before reducing and returning the results back to our driver node.